In [100]:
import torch
import torch.nn as nn


def gen_data(num_data=30, x_range=(-1, 1)):
    """数据集"""
    w = 1.5
    tr_x = torch.linspace(*x_range, num_data).unsqueeze_(1)
    tr_y = w * tr_x + torch.normal(0, 0.8, size=tr_x.size())
    te_x = torch.linspace(*x_range, num_data).unsqueeze_(1)
    te_y = w * te_x + torch.normal(0, 0.8, size=te_x.size())

    return tr_x, tr_y, te_x, te_y


train_x, train_y, test_x, test_y = gen_data(x_range=(-1, 1))


class MLP(nn.Module):
    def __init__(self, neural_num):
        super(MLP, self).__init__()
        self.linears = nn.Sequential(
            nn.Linear(1, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(),  # Dropout层
            nn.Linear(neural_num, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(neural_num, neural_num),
            nn.ReLU(inplace=True),

            nn.Dropout(),
            nn.Linear(neural_num, 1),
        )

    def forward(self, x):
        return self.linears(x)


net_drop = MLP(neural_num=200)  # 模型
optim_sgd = torch.optim.SGD(net_drop.parameters(), lr=0.01, momentum=0.9)  # 优化器
loss_func = torch.nn.MSELoss()  # 损失函数

In [101]:
for epoch in range(2000):
    train_pred = net_drop(test_x)
    loss = loss_func(train_pred, train_y)
    optim_sgd.zero_grad()
    loss.backward()
    optim_sgd.step()

    if (epoch + 1) % 200 == 0:
        # Sets the module in evaluation mode.(即设置模型及所有子孙模型的training属性为False)
        net_drop.eval()  # 影响Dropout和BatchNorm1d,BatchNorm2d,BatchNorm3d

        test_pred = net_drop(test_x)
        print('test loss:{}'.format(loss_func(test_pred, test_y)))

        # Sets the module in training mode.(即设置模型及所有子孙模型的training属性为True)
        net_drop.train()  # 影响Dropout和BatchNorm1d,BatchNorm2d,BatchNorm3d

test loss:0.36889684200286865
test loss:0.3744291365146637
test loss:0.3963598906993866
test loss:0.3911718428134918
test loss:0.3948180079460144
test loss:0.4014647901058197
test loss:0.38481712341308594
test loss:0.3931702971458435
test loss:0.4052799940109253
test loss:0.4079582989215851
